Examen : NANTCHOUANG GUEPNANG Nelly Kristelle

In [2]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [27]:
from pymongo import MongoClient
from pprint import pprint
import datetime

(a) Connexion à MongoDB via pymongo en utilisant nos credentials definis dans docker-compose

In [4]:
client = MongoClient(
    host="127.0.0.1",
    port = 27017,
    username = "",
    password = "",
    
)

(b) Liste des bases de données disponibles :

In [5]:
print(client.list_database_names())

['admin', 'config', 'local', 'sample', 'test']


(c) Liste des collections disponibles dans la base "sample"

In [7]:
sample = client["sample"]
print(sample.list_collection_names())

['zips', 'books']


(d) Affichons un Document de la Collection books

In [11]:
books = sample["books"]
pprint(books.find_one())

{'_id': 1,
 'authors': ['W. Frank Ableson', 'Charlie Collins', 'Robi Sen'],
 'categories': ['Open Source', 'Mobile'],
 'isbn': '1933988673',
 'longDescription': 'Android is an open source mobile phone platform based on '
                    'the Linux operating system and developed by the Open '
                    'Handset Alliance, a consortium of over 30 hardware, '
                    'software and telecom companies that focus on open '
                    'standards for mobile devices. Led by search giant, '
                    'Google, Android is designed to deliver a better and more '
                    'open and cost effective mobile experience.    Unlocking '
                    "Android: A Developer's Guide provides concise, hands-on "
                    'instruction for the Android operating system and '
                    'development tools. This book teaches important '
                    'architectural concepts in a straightforward writing style '
                    

(e) Affichons le Nombre de Documents dans cette Collection

In [15]:
print(books.count_documents({}))

431


## Exploration de la Base

 (a)  - Affichons le nombre de livres avec plus de 400 pages, affichons ensuite le nombre de livres ayant plus de 400 pages ET qui sont publiés.

In [16]:
#Nombre de livres avec plus de 400 pages
res = books.count_documents({"pageCount":{"$gt":400}})
print(f"Nombre de livres avec plus de 400 pages: {res}")

#"Nombre de livres avec plus de 400 pages et qui sont publies
req = books.count_documents({"pageCount":{"$gt":400}, "staus": "PUBLISH"})
print(f"Nombre de livres avec plus de 400 pages et sont publies: {req}")

Nombre de livres avec plus de 400 pages: 157
Nombre de livres avec plus de 400 pages et sont publies: 0


(b) Livres avec le Mot-Clé "Android" dans leur Description

In [18]:
req = books.count_documents({
    "$or": [
        {"shortDescription" : {"$regex": "Android", "$options": "i"}},
        {"longDescription": {"$regex": "Android", "$options": "i"}}
    ]
})

print(f"Nombre de livres avec mot-cle Android: {req}")

Nombre de livres avec mot-cle Android: 4


(c) Listes de Catégories Distinctes par Index

In [21]:
categories_lists = books.aggregate([
    {
        "$group": {
            "_id": None,
            "Catégorie1": {"$addToSet": {"$arrayElemAt": ["$categories", 0]}},
            "Catégorie2": {"$addToSet": {"$arrayElemAt": ["$categories", 1]}}
        }
    }
])

for result in categories_lists:
    print("Catégories 1:", result["Catégorie1"])
    print("Catégories 2:", result["Catégorie2"])


Catégories 1: ['SOA', 'Microsoft', 'Perl', 'Internet', 'Python', 'Algorithmic Art', 'Networking', 'java', 'Microsoft/.NET', 'Object-Oriented Programming', 'Software Engineering', 'Web Development', 'Open Source', 'Object-Technology Programming', 'Theory', 'Java', 'internet', 'Mobile Technology', 'In Action', 'Client-Server', 'Computer Graphics', 'Microsoft .NET', 'Next Generation Databases', 'PowerBuilder', 'PHP', 'Programming', 'Miscellaneous', 'Business', 'XML']
Catégories 2: ['Internet', 'Python', 'Software Engineering', 'Perl', 'Networking', 'Theory', 'Open Source', 'P', 'Business', '.NET', 'S', 'Java', 'Software Development', 'Computer Graphics', '', 'Mobile', 'Client-Server']


(d) Livres avec des Noms de Langages dans leur Description Longue

In [22]:
langages = ["Python", "Java", "C++", "Scala"]
pattern = "|".join(langages)  # Crée une expression régulière "Python|Java|C++|Scala"

books_with_langages = books.count_documents({
    "longDescription": {"$regex": pattern, "$options": "i"}
})

print(f"Nombre de livres contenant des noms de langages dans leur description longue: {books_with_langages}")


Nombre de livres contenant des noms de langages dans leur description longue: 263


(e) Informations Statistiques sur le Nombre de Pages par Catégorie

In [23]:
stats_pages_par_categorie = books.aggregate([
    {"$unwind": "$categories"},
    {"$group": {
        "_id": "$categories",
        "maxPages": {"$max": "$pageCount"},
        "minPages": {"$min": "$pageCount"},
        "avgPages": {"$avg": "$pageCount"}
    }},
    {"$sort": {"_id": 1}}
])

for stat in stats_pages_par_categorie:
    pprint(stat)


{'_id': '', 'avgPages': 284.0, 'maxPages': 368, 'minPages': 200}
{'_id': '.NET', 'avgPages': 925.0, 'maxPages': 925, 'minPages': 925}
{'_id': 'Algorithmic Art', 'avgPages': 0.0, 'maxPages': 0, 'minPages': 0}
{'_id': 'Business',
 'avgPages': 404.1666666666667,
 'maxPages': 696,
 'minPages': 180}
{'_id': 'Client Server', 'avgPages': 368.0, 'maxPages': 368, 'minPages': 368}
{'_id': 'Client-Server', 'avgPages': 516.0, 'maxPages': 1101, 'minPages': 220}
{'_id': 'Computer Graph', 'avgPages': 400.0, 'maxPages': 400, 'minPages': 400}
{'_id': 'Computer Graphics',
 'avgPages': 520.3333333333334,
 'maxPages': 840,
 'minPages': 328}
{'_id': 'In Action', 'avgPages': 0.0, 'maxPages': 0, 'minPages': 0}
{'_id': 'Internet',
 'avgPages': 441.2439024390244,
 'maxPages': 912,
 'minPages': 0}
{'_id': 'Java', 'avgPages': 412.21875, 'maxPages': 1088, 'minPages': 0}
{'_id': 'Microsoft', 'avgPages': 572.375, 'maxPages': 925, 'minPages': 344}
{'_id': 'Microsoft .NET',
 'avgPages': 437.02941176470586,
 'maxPages

(f) Extraction de l'Année, du Mois et du Jour de la Date de Publication

In [28]:
pipeline_f = [
    {"$match": {"publishedDate": {"$gt": datetime.datetime(2009, 1, 1)}}},
    {"$project": {
        "year": {"$year": "$publishedDate"},
        "month": {"$month": "$publishedDate"},
        "day": {"$dayOfMonth": "$publishedDate"},
        "title": 1  # Inclut le titre pour référence
    }},
    {"$limit": 20}
]

results_f = books.aggregate(pipeline_f)
for result in results_f:
    pprint(result)


{'_id': 1, 'day': 1, 'month': 4, 'title': 'Unlocking Android', 'year': 2009}
{'_id': 2,
 'day': 14,
 'month': 1,
 'title': 'Android in Action, Second Edition',
 'year': 2011}
{'_id': 3,
 'day': 3,
 'month': 6,
 'title': 'Specification by Example',
 'year': 2011}
{'_id': 4, 'day': 2, 'month': 2, 'title': 'Flex 3 in Action', 'year': 2009}
{'_id': 5, 'day': 15, 'month': 11, 'title': 'Flex 4 in Action', 'year': 2010}
{'_id': 8, 'day': 15, 'month': 10, 'title': 'Flex on Java', 'year': 2010}
{'_id': 9, 'day': 4, 'month': 6, 'title': 'Griffon in Action', 'year': 2012}
{'_id': 10, 'day': 12, 'month': 12, 'title': 'OSGi in Depth', 'year': 2011}
{'_id': 13, 'day': 1, 'month': 11, 'title': 'Hello! Flex 4', 'year': 2009}
{'_id': 16,
 'day': 16,
 'month': 4,
 'title': 'Brownfield Application Development in .NET',
 'year': 2010}
{'_id': 17, 'day': 12, 'month': 12, 'title': 'MongoDB in Action', 'year': 2011}
{'_id': 30,
 'day': 1,
 'month': 6,
 'title': 'jQuery in Action, Second Edition',
 'year': 20

(g) Création de Nouveaux Attributs pour les Auteurs

In [29]:
pipeline_g = [
    {"$project": {
        "author1": {"$arrayElemAt": ["$authors", 0]},
        "author2": {"$arrayElemAt": ["$authors", 1]},
        "title": 1  # Inclut le titre pour référence
    }},
    {"$sort": {"publishedDate": 1}},
    {"$limit": 20}
]

results_g = books.aggregate(pipeline_g)
for result in results_g:
    pprint(result)


{'_id': 20,
 'author1': 'Michael J. Barlotta',
 'author2': 'Jason R. Weiss',
 'title': 'Taming Jaguar'}
{'_id': 11, 'author1': 'Peter Armstrong', 'title': 'Flexible Rails'}
{'_id': 5,
 'author1': 'Tariq Ahmed',
 'author2': 'Dan Orlando',
 'title': 'Flex 4 in Action'}
{'_id': 18,
 'author1': 'Michael J. Barlotta',
 'title': 'Distributed Application Development with PowerBuilder 6.0'}
{'_id': 14,
 'author1': 'Levi Asher',
 'author2': 'Christian Crumlish',
 'title': 'Coffeehouse'}
{'_id': 9,
 'author1': 'Andres Almiray',
 'author2': 'Danno Ferrin',
 'title': 'Griffon in Action'}
{'_id': 21,
 'author1': 'Jon Barrilleaux',
 'title': '3D User Interfaces with Java 3D'}
{'_id': 2,
 'author1': 'W. Frank Ableson',
 'author2': 'Robi Sen',
 'title': 'Android in Action, Second Edition'}
{'_id': 13, 'author1': 'Peter Armstrong', 'title': 'Hello! Flex 4'}
{'_id': 6,
 'author1': 'Satnam Alag',
 'title': 'Collective Intelligence in Action'}
{'_id': 17, 'author1': 'Kyle Banker', 'title': 'MongoDB in Act

(h) Nombre de Publications par le Premier Auteur

In [30]:
pipeline_h = [
    {"$project": {
        "firstAuthor": {"$arrayElemAt": ["$authors", 0]}
    }},
    {"$group": {
        "_id": "$firstAuthor",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

results_h = books.aggregate(pipeline_h)
for result in results_h:
    pprint(result)


{'_id': None, 'count': 37}
{'_id': 'Vikram Goyal', 'count': 12}
{'_id': 'Don Jones', 'count': 6}
{'_id': 'Christian Bauer', 'count': 4}
{'_id': 'Craig Walls', 'count': 4}
{'_id': 'W. Frank Ableson', 'count': 3}
{'_id': 'Richard Siddaway', 'count': 3}
{'_id': 'Dave Crane', 'count': 3}
{'_id': 'Tim Hatton', 'count': 3}
{'_id': 'Ted Neward', 'count': 3}


(i) Distribution du Nombre d'Auteurs

In [31]:
pipeline_i = [
    {"$project": {
        "authorsCount": {"$size": "$authors"}
    }},
    {"$group": {
        "_id": "$authorsCount",
        "count": {"$sum": 1}
    }},
    {"$sort": {"_id": 1}}
]

results_i = books.aggregate(pipeline_i)
for result in results_i:
    pprint(result)


{'_id': 0, 'count': 37}
{'_id': 1, 'count': 206}
{'_id': 2, 'count': 105}
{'_id': 3, 'count': 9}
{'_id': 4, 'count': 47}
{'_id': 5, 'count': 16}
{'_id': 6, 'count': 6}
{'_id': 7, 'count': 2}
{'_id': 8, 'count': 3}
